In [1]:
# 导入库
import os
from pathlib import Path
os.chdir(Path(os.getcwd()).parent.parent)

In [2]:
import pandas as pd
from data_manager.DailyDataManager import DailyDataManager
from data_manager.BasicsDataManager import BasicsDataManager

In [3]:
self = DailyDataManager()
daily_data = self.daily_data

In [4]:
daily_volume = daily_data[daily_data.datetime.dt.year==2020].groupby(['datetime', 'underlying_symbol'])['volume'].sum().unstack(level=-1)
daily_volume.head()

underlying_symbol,A,AG,AL,AP,AU,B,BB,BC,BU,C,...,TA,TF,TS,UR,V,WH,WR,Y,ZC,ZN
datetime,,,,,,,,,,,,,,,,,,,,,
2020-01-02,33138.0,427012.0,67718.0,98371.0,81747.0,105380.0,0.0,NaN,161429.0,235159.0,...,540352.0,11845.0,19215.0,23107.0,58181.0,5.0,4.0,357395.0,77687.0,138349.0
2020-01-03,44333.0,1214044.0,144202.0,156669.0,261163.0,98297.0,0.0,NaN,312681.0,305615.0,...,728476.0,12596.0,23093.0,27089.0,76283.0,1.0,1.0,683952.0,116181.0,214259.0
2020-01-06,51395.0,1184488.0,124343.0,180630.0,318442.0,85778.0,0.0,NaN,438971.0,244075.0,...,915883.0,11054.0,16210.0,26461.0,181880.0,0.0,16.0,503226.0,102369.0,315341.0
2020-01-07,59706.0,2038735.0,105737.0,115331.0,548992.0,47762.0,0.0,NaN,290182.0,412823.0,...,777582.0,10491.0,22755.0,30521.0,90157.0,0.0,0.0,556208.0,54972.0,194636.0
2020-01-08,57122.0,1326716.0,102303.0,134602.0,387893.0,56731.0,0.0,NaN,401760.0,351171.0,...,979840.0,10429.0,16674.0,49595.0,120119.0,0.0,13.0,591960.0,90429.0,357467.0


In [5]:
mean_volume = daily_volume.mean(axis=0)
mean_volume.head()

underlying_symbol
A     2.446158e+05
AG    1.480709e+06
AL    2.176308e+05
AP    2.592974e+05
AU    2.184655e+05
dtype: float64

In [6]:
mean_volume = mean_volume.to_frame("volume").reset_index()

In [7]:
self = BasicsDataManager()
all_instruments = self.get_all_instruments()
symbol_exchange_df = all_instruments[['underlying_symbol', 'exchange']].drop_duplicates()

In [8]:
symbol_exchange_df.head()

,underlying_symbol,exchange
4341,A,DCE
111,AG,SHFE
218,AL,SHFE
477,AP,CZCE
499,AU,SHFE


In [9]:
mean_volume = pd.merge(left=mean_volume, right=symbol_exchange_df, on='underlying_symbol', how='left')
mean_volume.loc[mean_volume.exchange!='CZCE', 'underlying_symbol'] = mean_volume.loc[mean_volume.exchange!='CZCE', 'underlying_symbol'].str.lower()

In [10]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [11]:
mean_volume.head()

,underlying_symbol,volume,exchange
0,a,244615.82,DCE
1,ag,1480708.68,SHFE
2,al,217630.80,SHFE
3,AP,259297.44,CZCE
4,au,218465.47,SHFE


In [12]:
mean_volume.sort_values(by='volume', ascending=False).set_index('underlying_symbol')['volume'].to_csv("期货品种2020年日平均成交量.csv")